# Data Preparation

## Why do this step?

- The data we scraped from the UFC website is raw data. Every row contains information about a fight that took place i.e. details like how many strikes were thrown in that fight and who won the fight.
- To prepare the data for prediction, every row can contain only an accurate representation of what each fighter has done in fights up until that fight! No data that was recorded during the fight can be present in that row.
- Our Target variable is Winner. The task has to be to predict the winner from the data available of each fighter up until the fight

## Looking at the data

In [8]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('../data/raw_total_fight_data.csv', sep=';')
fighter_details = pd.read_csv('../data/raw_fighter_details.csv', index_col='fighter_name')

In [9]:
df.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Jack Della Maddalena,Randy Brown,1,0,14 of 31,11 of 39,45%,28%,18 of 35,11 of 39,...,0 of 0,Submission,1,2:13,3 Rnd (5-5-5),Marc Goddard,"February 11, 2023","Perth, Western Australia, Australia",Welterweight Bout,Jack Della Maddalena
1,Tyson Pedro,Modestas Bukauskas,0,0,45 of 81,44 of 109,55%,40%,54 of 90,56 of 121,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Steve Perceval,"February 11, 2023","Perth, Western Australia, Australia",Light Heavyweight Bout,Modestas Bukauskas
2,Shannon Ross,Kleydson Rodrigues,0,1,1 of 4,14 of 22,25%,63%,1 of 4,16 of 24,...,0 of 1,KO/TKO,1,0:59,3 Rnd (5-5-5),Matt Wynne,"February 11, 2023","Perth, Western Australia, Australia",Flyweight Bout,Kleydson Rodrigues
3,Justin Tafa,Parker Porter,1,0,8 of 16,7 of 16,50%,43%,8 of 16,7 of 16,...,0 of 0,KO/TKO,1,1:06,3 Rnd (5-5-5),Jason Herzog,"February 11, 2023","Perth, Western Australia, Australia",Heavyweight Bout,Justin Tafa
4,Josh Culibao,Melsik Baghdasaryan,0,0,16 of 45,26 of 44,35%,59%,16 of 45,26 of 44,...,0 of 0,Submission,2,2:02,3 Rnd (5-5-5),Peter Hickmott,"February 11, 2023","Perth, Western Australia, Australia",Featherweight Bout,Josh Culibao


In [10]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,6956,6957,6958,6959,6960,6961,6962,6963,6964,6965
R_fighter,Jack Della Maddalena,Tyson Pedro,Shannon Ross,Justin Tafa,Josh Culibao,Jimmy Crute,Islam Makhachev,Yair Rodriguez,Shane Young,Loma Lookboonmee,...,Johnny Rhodes,Royce Gracie,Patrick Smith,Orlando Wiet,Johnny Rhodes,Jason DeLucia,Remco Pardoel,Frank Hamaker,Scott Morris,Patrick Smith
B_fighter,Randy Brown,Modestas Bukauskas,Kleydson Rodrigues,Parker Porter,Melsik Baghdasaryan,Alonzo Menifield,Alexander Volkanovski,Josh Emmett,Blake Bilder,Elise Reed,...,Fred Ettish,Minoki Ichihara,Scott Morris,Robert Lucarelli,David Levicki,Scott Baker,Alberta Cerra Leon,Thaddeus Luster,Sean Daugherty,Ray Wizard
R_KD,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,0,1,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR.,14 of 31,45 of 81,1 of 4,8 of 16,16 of 45,31 of 62,57 of 95,62 of 107,80 of 196,17 of 27,...,13 of 29,2 of 4,13 of 17,8 of 12,11 of 17,3 of 5,4 of 6,2 of 3,1 of 1,1 of 1
B_SIG_STR.,11 of 39,44 of 109,14 of 22,7 of 16,26 of 44,56 of 83,70 of 143,22 of 52,111 of 190,17 of 29,...,4 of 7,3 of 7,0 of 0,2 of 6,4 of 5,0 of 2,1 of 3,0 of 0,0 of 4,1 of 1
R_SIG_STR_pct,45%,55%,25%,50%,35%,50%,60%,57%,40%,62%,...,44%,50%,76%,66%,64%,60%,66%,66%,100%,100%
B_SIG_STR_pct,28%,40%,63%,43%,59%,67%,48%,42%,58%,58%,...,57%,42%,---,33%,80%,0%,33%,---,0%,100%
R_TOTAL_STR.,18 of 35,54 of 90,1 of 4,8 of 16,16 of 45,62 of 103,95 of 135,98 of 143,81 of 197,22 of 34,...,21 of 38,110 of 114,19 of 25,11 of 15,74 of 86,20 of 25,20 of 22,14 of 15,2 of 2,1 of 1
B_TOTAL_STR.,11 of 39,56 of 121,16 of 24,7 of 16,26 of 44,99 of 127,164 of 255,35 of 72,126 of 206,41 of 60,...,7 of 11,12 of 16,0 of 0,2 of 6,95 of 102,14 of 23,9 of 11,0 of 0,1 of 5,2 of 2


In [11]:
df.describe()

,R_KD,B_KD,R_SUB_ATT,B_SUB_ATT,R_REV,B_REV,last_round
count,6966.000000,6966.000000,6966.000000,6966.000000,6966.000000,6966.000000,6966.000000
mean,0.248923,0.183175,0.461384,0.333190,0.136520,0.135515,2.333764
std,0.524814,0.461943,0.903619,0.775585,0.428552,0.419563,1.013973
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000
max,5.000000,4.000000,10.000000,7.000000,6.000000,4.000000,5.000000


In [12]:
df.dtypes

R_fighter          object
B_fighter          object
R_KD                int64
B_KD                int64
R_SIG_STR.         object
B_SIG_STR.         object
R_SIG_STR_pct      object
B_SIG_STR_pct      object
R_TOTAL_STR.       object
B_TOTAL_STR.       object
R_TD               object
B_TD               object
R_TD_pct           object
B_TD_pct           object
R_SUB_ATT           int64
B_SUB_ATT           int64
R_REV               int64
B_REV               int64
R_CTRL             object
B_CTRL             object
R_HEAD             object
B_HEAD             object
R_BODY             object
B_BODY             object
R_LEG              object
B_LEG              object
R_DISTANCE         object
B_DISTANCE         object
R_CLINCH           object
B_CLINCH           object
R_GROUND           object
B_GROUND           object
win_by             object
last_round          int64
last_round_time    object
Format             object
Referee            object
date               object
location    

In [13]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR.', 'B_SIG_STR.',
       'R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'R_CTRL', 'B_CTRL', 'R_HEAD', 'B_HEAD', 'R_BODY',
       'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH',
       'B_CLINCH', 'R_GROUND', 'B_GROUND', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner'],
      dtype='object')

### Column definitions:

- `R_` and `B_` prefix signifies red and blue corner fighter stats respectively
- `KD` is number of knockdowns
- `SIG_STR` is no. of significant strikes 'landed of attempted'
- `SIG_STR_pct` is significant strikes percentage
- `TOTAL_STR` is total strikes 'landed of attempted'
- `TD` is no. of takedowns
- `TD_pct` is takedown percentages
- `SUB_ATT` is no. of submission attempts
- `PASS` is no. times the guard was passed?
- `REV` is the no. of reversals
- `CTRL` is the time spent with ground control
- `HEAD` is no. of significant strinks to the head 'landed of attempted'
- `BODY` is no. of significant strikes to the body 'landed of attempted'
- `CLINCH` is no. of significant strikes in the clinch 'landed of attempted'
- `GROUND` is no. of significant strikes on the ground 'landed of attempted'
- `win_by` is method of win
- `last_round` is last round of the fight (ex. if it was a KO in 1st, then this will be 1)
- `last_round_time` is when the fight ended in the last round
- `Format` is the format of the fight (3 rounds, 5 rounds etc.)
- `Referee` is the name of the Ref
- `date` is the date of the fight
- `location` is the location in which the event took place
- `Fight_type` is which weight class and whether it's a title bout or not
- `Winner` is the winner of the fight

#### Per fighter career wide stats
- `SLpM` - Significant Strikes Landed per Minute
- `Str_Acc.` - Significant Striking Accuracy
- `SApM` - Significant Strikes Absorbed per Minute
- `Str_Def` - Significant Strike Defence (the % of opponents strikes that did not land)
- `TD_Avg` - Average Takedowns Landed per 15 minutes
- `TD_Acc` - Takedown Accuracy
- `TD_Def` - Takedown Defense (the % of opponents TD attempts that did not land)
- `Sub_Avg` - Average Submissions Attempted per 15 minutes 

## Todo:

- Separate `landed of attempted` to separate columns

- Convert `Fight_type` into two separate columns, `weight_class` and `Title_fight` (True or False)

- Convert `last_round_time` to `total_time_fought` by using `last_round` and `Format`

- Convert `CTRL` to `time_in_CTRL`

- Convert percentages to fractions

- Since the data is a description of each fight, we have to convert it into a format that shows the compilation data of each fighter up until that fight. This means every row will look a lot different than it looks now.

- Create `current_win_streak`, `current_lose_streak`, `longest_win_streak`, `wins`, `losses`, `draw`

- Create fighter `height`, `reach`, `weight`, `age`

In [14]:
fighter_details.drop(
    columns=["SLpM",
            "Str_Acc",
            "SApM",
            "Str_Def",
            "TD_Avg",
            "TD_Acc",
            "TD_Def",
            "Sub_Avg",
        ], inplace=True)

### Splitting landed of attempted to different columns

In [15]:
columns = ['R_SIG_STR.', 'B_SIG_STR.', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_HEAD', 'B_HEAD', 'R_BODY','B_BODY', 'R_LEG', 'B_LEG', 
        'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH','B_CLINCH', 'R_GROUND', 'B_GROUND']

for column in columns:
    print(f"{column} data type is: {df[column].dtype}")

R_SIG_STR. data type is: object
B_SIG_STR. data type is: object
R_TOTAL_STR. data type is: object
B_TOTAL_STR. data type is: object
R_TD data type is: object
B_TD data type is: object
R_HEAD data type is: object
B_HEAD data type is: object
R_BODY data type is: object
B_BODY data type is: object
R_LEG data type is: object
B_LEG data type is: object
R_DISTANCE data type is: object
B_DISTANCE data type is: object
R_CLINCH data type is: object
B_CLINCH data type is: object
R_GROUND data type is: object
B_GROUND data type is: object


In [16]:
attempt_suffix = '_att'
landed_suffix = '_landed'

for column in columns:
    df[column+attempt_suffix] = df[column].apply(lambda X: int(X.split('of')[1]))
    df[column+landed_suffix] = df[column].apply(lambda X: int(X.split('of')[0]))
    
df.drop(columns, axis=1, inplace=True)

In [17]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'R_CTRL', 'B_CTRL', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner', 'R_SIG_STR._att', 'R_SIG_STR._landed',
       'B_SIG_STR._att', 'B_SIG_STR._landed', 'R_TOTAL_STR._att',
       'R_TOTAL_STR._landed', 'B_TOTAL_STR._att', 'B_TOTAL_STR._landed',
       'R_TD_att', 'R_TD_landed', 'B_TD_att', 'B_TD_landed', 'R_HEAD_att',
       'R_HEAD_landed', 'B_HEAD_att', 'B_HEAD_landed', 'R_BODY_att',
       'R_BODY_landed', 'B_BODY_att', 'B_BODY_landed', 'R_LEG_att',
       'R_LEG_landed', 'B_LEG_att', 'B_LEG_landed', 'R_DISTANCE_att',
       'R_DISTANCE_landed', 'B_DISTANCE_att', 'B_DISTANCE_landed',
       'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att', 'B_CLINCH_landed',
       'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att', 'B_GROUND_landed'],
      dtype

### Replacing Winner NaNs as Draw

In [18]:
for column in df.columns:
    if df[column].isnull().sum() != 0:
        print(f"NaN values in {column} = {df[column].isnull().sum()}")

NaN values in Referee = 32
NaN values in Winner = 125


* 83 missing values in winner and 23 missing values in Referee

In [19]:
df[df['Winner'].isnull()]['win_by'].value_counts()

Overturned              50
Decision - Majority     30
Could Not Continue      20
Decision - Split        16
Decision - Unanimous     6
Other                    3
Name: win_by, dtype: int64

* Here, Overturned means due to drug test being positive and Could not Continue means there was an illegal blow which was not enough to be disqualified but the fighter could not continue.
* The rest are different forms of draw

* Replacing all of these with draw

In [20]:
df['Winner'].fillna('Draw', inplace=True)

### Converting percentages to fractions

In [21]:
pct_columns = ['R_SIG_STR_pct','B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct']

def pct_to_frac(X):
    if X != '---':
        return float(X.replace('%', ''))/100
    else:
        # if '---' means it's taking pct of `0 of 0`. 
        # Taking a call here to consider 0 landed of 0 attempted as 0 percentage
        return 0

for column in pct_columns:
    df[column] = df[column].apply(pct_to_frac)

In [22]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,6956,6957,6958,6959,6960,6961,6962,6963,6964,6965
R_fighter,Jack Della Maddalena,Tyson Pedro,Shannon Ross,Justin Tafa,Josh Culibao,Jimmy Crute,Islam Makhachev,Yair Rodriguez,Shane Young,Loma Lookboonmee,...,Johnny Rhodes,Royce Gracie,Patrick Smith,Orlando Wiet,Johnny Rhodes,Jason DeLucia,Remco Pardoel,Frank Hamaker,Scott Morris,Patrick Smith
B_fighter,Randy Brown,Modestas Bukauskas,Kleydson Rodrigues,Parker Porter,Melsik Baghdasaryan,Alonzo Menifield,Alexander Volkanovski,Josh Emmett,Blake Bilder,Elise Reed,...,Fred Ettish,Minoki Ichihara,Scott Morris,Robert Lucarelli,David Levicki,Scott Baker,Alberta Cerra Leon,Thaddeus Luster,Sean Daugherty,Ray Wizard
R_KD,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,0,1,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR_pct,0.45,0.55,0.25,0.5,0.35,0.5,0.6,0.57,0.4,0.62,...,0.44,0.5,0.76,0.66,0.64,0.6,0.66,0.66,1,1
B_SIG_STR_pct,0.28,0.4,0.63,0.43,0.59,0.67,0.48,0.42,0.58,0.58,...,0.57,0.42,0,0.33,0.8,0,0.33,0,0,1
R_TD_pct,0,0.66,0,0,0,0.5,0.44,0,0,0.66,...,0,1,0,0,1,0,1,1,1,0
B_TD_pct,0,0,0,0,0,0,0,0.25,0.14,0,...,0,0,0,1,0,1,0,0,0,0
R_SUB_ATT,1,0,0,0,1,3,0,1,0,1,...,1,2,0,0,0,5,1,3,1,1
B_SUB_ATT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Creating a title_bout feature and weight_class

In [23]:
df['Fight_type'].value_counts()

Lightweight Bout                                                     1169
Welterweight Bout                                                    1138
Middleweight Bout                                                     884
Featherweight Bout                                                    634
Heavyweight Bout                                                      598
                                                                     ... 
Ultimate Fighter 11 Middleweight Tournament Title Bout                  1
Ultimate Fighter 1 Middleweight Tournament Title Bout                   1
Ultimate Fighter 19 Light Heavyweight Tournament Title Bout             1
Ultimate Fighter Latin America Bantamweight Tournament Title Bout       1
Ultimate Fighter Brazil 4 Bantamweight Tournament Title Bout            1
Name: Fight_type, Length: 109, dtype: int64

In [24]:
df['Fight_type'].value_counts()[df['Fight_type'].value_counts() > 1].index

Index(['Lightweight Bout', 'Welterweight Bout', 'Middleweight Bout',
       'Featherweight Bout', 'Heavyweight Bout', 'Light Heavyweight Bout',
       'Bantamweight Bout', 'Flyweight Bout', 'Women's Strawweight Bout',
       'Women's Flyweight Bout', 'Women's Bantamweight Bout',
       'Open Weight Bout', 'Catch Weight Bout',
       'UFC Light Heavyweight Title Bout', 'UFC Welterweight Title Bout',
       'UFC Heavyweight Title Bout', 'UFC Middleweight Title Bout',
       'UFC Lightweight Title Bout', 'UFC Flyweight Title Bout',
       'Women's Featherweight Bout', 'UFC Featherweight Title Bout',
       'UFC Bantamweight Title Bout', 'UFC Women's Strawweight Title Bout',
       'UFC Women's Bantamweight Title Bout',
       'UFC Women's Flyweight Title Bout',
       'UFC Interim Heavyweight Title Bout',
       'UFC Women's Featherweight Title Bout',
       'UFC Superfight Championship Bout',
       'UFC Interim Bantamweight Title Bout',
       'UFC Interim Featherweight Title Bout',
   

In [25]:
df['title_bout'] = df['Fight_type'].apply(lambda X: True if 'Title Bout' in X else False)

In [26]:
def make_weight_class(X):
    for weight_class in weight_classes:
        if weight_class in X:
            return weight_class
    if X == 'Catch Weight Bout' or 'Catchweight Bout':
        return 'Catch Weight'
    else:
        return 'Open Weight'

In [27]:
weight_classes = ['Women\'s Strawweight', 'Women\'s Bantamweight', 
                  'Women\'s Featherweight', 'Women\'s Flyweight', 'Lightweight', 
                  'Welterweight', 'Middleweight','Light Heavyweight', 
                  'Heavyweight', 'Featherweight','Bantamweight', 'Flyweight', 'Open Weight']

df['weight_class'] = df['Fight_type'].apply(make_weight_class)

In [28]:
df[df['weight_class'].isnull()]['Fight_type'].value_counts()

Series([], Name: Fight_type, dtype: int64)

### Creating total_time_fought

In [29]:
df['Format'].value_counts()

3 Rnd (5-5-5)           6136
5 Rnd (5-5-5-5-5)        606
1 Rnd + OT (12-3)         80
3 Rnd + OT (5-5-5-5)      30
No Time Limit             29
1 Rnd (20)                21
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                 8
1 Rnd (10)                 6
1 Rnd (12)                 4
1 Rnd + OT (30-5)          3
1 Rnd + OT (15-3)          2
1 Rnd (18)                 2
1 Rnd + OT (27-3)          1
1 Rnd (30)                 1
1 Rnd + 2OT (24-3-3)       1
1 Rnd + OT (30-3)          1
1 Rnd + OT (31-5)          1
Name: Format, dtype: int64

In [30]:
df['Format'].value_counts().index

Index(['3 Rnd (5-5-5)', '5 Rnd (5-5-5-5-5)', '1 Rnd + OT (12-3)',
       '3 Rnd + OT (5-5-5-5)', 'No Time Limit', '1 Rnd (20)',
       '1 Rnd + 2OT (15-3-3)', '2 Rnd (5-5)', '1 Rnd (15)', '1 Rnd (10)',
       '1 Rnd (12)', '1 Rnd + OT (30-5)', '1 Rnd + OT (15-3)', '1 Rnd (18)',
       '1 Rnd + OT (27-3)', '1 Rnd (30)', '1 Rnd + 2OT (24-3-3)',
       '1 Rnd + OT (30-3)', '1 Rnd + OT (31-5)'],
      dtype='object')

In [31]:
time_in_first_round = {'3 Rnd (5-5-5)': 5*60, '5 Rnd (5-5-5-5-5)': 5*60, '1 Rnd + OT (12-3)': 12*60,
       'No Time Limit': 1, '3 Rnd + OT (5-5-5-5)': 5*60, '1 Rnd (20)': 1*20,
       '2 Rnd (5-5)': 5*60, '1 Rnd (15)': 15*60, '1 Rnd (10)': 10*60,
       '1 Rnd (12)':12*60, '1 Rnd + OT (30-5)': 30*60, '1 Rnd (18)': 18*60, '1 Rnd + OT (15-3)': 15*60,
       '1 Rnd (30)': 30*60, '1 Rnd + OT (31-5)': 31*5,
       '1 Rnd + OT (27-3)': 27*60, '1 Rnd + OT (30-3)': 30*60}

exception_format_time = {'1 Rnd + 2OT (15-3-3)': [15*60, 3*60], '1 Rnd + 2OT (24-3-3)': [24*60, 3*60]}

# '1 Rnd + 2OT (15-3-3)' and '1 Rnd + 2OT (24-3-3)' is not included because it has 3 uneven timed rounds. 
# We'll have to deal with it separately

In [32]:
# Converting to seconds
df['last_round_time'] = df['last_round_time'].apply(lambda X: int(X.split(':')[0])*60 + int(X.split(':')[1]))

In [33]:
def get_total_time(row):
    if row['Format'] in time_in_first_round.keys():
        return (row['last_round'] - 1) * time_in_first_round[row['Format']] + row['last_round_time']
    elif row['Format'] in exception_format_time.keys():
        if (row['last_round'] - 1) >= 2:
            return exception_format_time[row['Format']][0] + (row['last_round'] - 2) * \
                    exception_format_time[row['Format']][1] + row['last_round_time']
        else:
            return (row['last_round'] - 1) * exception_format_time[row['Format']][0] + row['last_round_time']
    
# So if the fight ended in round 1, we only need last_round_time. 
# If it ended in round 2, we need the full time of round 1 and the last_round_time
# This works for fights with same time in each round and fights with only two rounds.

In [34]:
df['total_time_fought(seconds)'] = df.apply(get_total_time, axis=1)

In [35]:
def get_no_of_rounds(X):
    if X == 'No Time Limit':
        return 1
    else:
        return len(X.split('(')[1].replace(')', '').split('-'))

df['no_of_rounds'] = df['Format'].apply(get_no_of_rounds)

In [36]:
df

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,...,B_CLINCH_att,B_CLINCH_landed,R_GROUND_att,R_GROUND_landed,B_GROUND_att,B_GROUND_landed,title_bout,weight_class,total_time_fought(seconds),no_of_rounds
0,Jack Della Maddalena,Randy Brown,1,0,0.45,0.28,0.00,0.0,1,0,...,0,0,11,7,0,0,False,Welterweight,133,3
1,Tyson Pedro,Modestas Bukauskas,0,0,0.55,0.40,0.66,0.0,0,0,...,14,10,1,1,0,0,False,Light Heavyweight,900,3
2,Shannon Ross,Kleydson Rodrigues,0,1,0.25,0.63,0.00,0.0,0,0,...,7,5,0,0,1,0,False,Flyweight,59,3
3,Justin Tafa,Parker Porter,1,0,0.50,0.43,0.00,0.0,0,0,...,1,0,0,0,0,0,False,Heavyweight,66,3
4,Josh Culibao,Melsik Baghdasaryan,0,0,0.35,0.59,0.00,0.0,1,0,...,3,2,0,0,0,0,False,Featherweight,422,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6961,Jason DeLucia,Scott Baker,0,0,0.60,0.00,0.00,1.0,5,0,...,0,0,1,1,0,0,False,Open Weight,401,1
6962,Remco Pardoel,Alberta Cerra Leon,0,0,0.66,0.33,1.00,0.0,1,0,...,0,0,5,4,2,0,False,Open Weight,591,1
6963,Frank Hamaker,Thaddeus Luster,0,0,0.66,0.00,1.00,0.0,3,0,...,0,0,2,1,0,0,False,Open Weight,292,1
6964,Scott Morris,Sean Daugherty,0,0,1.00,0.00,1.00,0.0,1,0,...,1,0,0,0,0,0,False,Open Weight,20,1


In [37]:
df.drop(['Format', 'Fight_type', 'last_round_time'], axis = 1, inplace=True)

### Create CTRL_time(seconds)

In [38]:
CTRL_columns = ['R_CTRL','B_CTRL']

def conv_to_sec(X):
    if X != '--':
        return int(X.split(':')[0])*60 + int(X.split(':')[1])
    else:
        # if '--' means there was no time spent on the ground. 
        # Taking a call here to consider this as 0 seconds
        return 0

for column in CTRL_columns:
    df[column+'_time(seconds)'] = df[column].apply(conv_to_sec)

In [39]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,6956,6957,6958,6959,6960,6961,6962,6963,6964,6965
R_fighter,Jack Della Maddalena,Tyson Pedro,Shannon Ross,Justin Tafa,Josh Culibao,Jimmy Crute,Islam Makhachev,Yair Rodriguez,Shane Young,Loma Lookboonmee,...,Johnny Rhodes,Royce Gracie,Patrick Smith,Orlando Wiet,Johnny Rhodes,Jason DeLucia,Remco Pardoel,Frank Hamaker,Scott Morris,Patrick Smith
B_fighter,Randy Brown,Modestas Bukauskas,Kleydson Rodrigues,Parker Porter,Melsik Baghdasaryan,Alonzo Menifield,Alexander Volkanovski,Josh Emmett,Blake Bilder,Elise Reed,...,Fred Ettish,Minoki Ichihara,Scott Morris,Robert Lucarelli,David Levicki,Scott Baker,Alberta Cerra Leon,Thaddeus Luster,Sean Daugherty,Ray Wizard
R_KD,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,0,1,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR_pct,0.45,0.55,0.25,0.5,0.35,0.5,0.6,0.57,0.4,0.62,...,0.44,0.5,0.76,0.66,0.64,0.6,0.66,0.66,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
weight_class,Welterweight,Light Heavyweight,Flyweight,Heavyweight,Featherweight,Light Heavyweight,Lightweight,Featherweight,Featherweight,Women's Strawweight,...,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight
total_time_fought(seconds),133,900,59,66,422,900,1500,559,900,344,...,187,308,30,170,733,401,591,292,20,58
no_of_rounds,3,3,3,3,3,3,5,5,3,3,...,1,1,1,1,1,1,1,1,1,1
R_CTRL_time(seconds),15,181,0,0,14,475,457,0,3,36,...,0,0,0,0,0,0,0,0,0,0


In [40]:
df.drop(['R_CTRL','B_CTRL'], axis = 1, inplace=True)

### Create another DataFrame to save the compiled data per fighter (Our Prediction DataFrame)

In [41]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'win_by', 'last_round', 'Referee', 'date', 'location',
       'Winner', 'R_SIG_STR._att', 'R_SIG_STR._landed', 'B_SIG_STR._att',
       'B_SIG_STR._landed', 'R_TOTAL_STR._att', 'R_TOTAL_STR._landed',
       'B_TOTAL_STR._att', 'B_TOTAL_STR._landed', 'R_TD_att', 'R_TD_landed',
       'B_TD_att', 'B_TD_landed', 'R_HEAD_att', 'R_HEAD_landed', 'B_HEAD_att',
       'B_HEAD_landed', 'R_BODY_att', 'R_BODY_landed', 'B_BODY_att',
       'B_BODY_landed', 'R_LEG_att', 'R_LEG_landed', 'B_LEG_att',
       'B_LEG_landed', 'R_DISTANCE_att', 'R_DISTANCE_landed', 'B_DISTANCE_att',
       'B_DISTANCE_landed', 'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att',
       'B_CLINCH_landed', 'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att',
       'B_GROUND_landed', 'title_bout', 'weight_class',
       'total_time_fought(seconds)', 'no_of_rounds',

In [42]:
df2 = df.copy()

In [43]:
df2.drop(['R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_CTRL_time(seconds)', 'B_CTRL_time(seconds)', 'R_REV', 'B_REV', 'win_by', 'last_round', 
        'R_SIG_STR._att', 'R_SIG_STR._landed',
       'B_SIG_STR._att', 'B_SIG_STR._landed', 'R_TOTAL_STR._att',
       'R_TOTAL_STR._landed', 'B_TOTAL_STR._att', 'B_TOTAL_STR._landed',
       'R_TD_att', 'R_TD_landed', 'B_TD_att', 'B_TD_landed', 'R_HEAD_att',
       'R_HEAD_landed', 'B_HEAD_att', 'B_HEAD_landed', 'R_BODY_att',
       'R_BODY_landed', 'B_BODY_att', 'B_BODY_landed', 'R_LEG_att',
       'R_LEG_landed', 'B_LEG_att', 'B_LEG_landed', 'R_DISTANCE_att',
       'R_DISTANCE_landed', 'B_DISTANCE_att', 'B_DISTANCE_landed',
       'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att', 'B_CLINCH_landed',
       'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att', 'B_GROUND_landed',
        'total_time_fought(seconds)'], axis = 1, inplace=True)
df2

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds
0,Jack Della Maddalena,Randy Brown,Marc Goddard,"February 11, 2023","Perth, Western Australia, Australia",Jack Della Maddalena,False,Welterweight,3
1,Tyson Pedro,Modestas Bukauskas,Steve Perceval,"February 11, 2023","Perth, Western Australia, Australia",Modestas Bukauskas,False,Light Heavyweight,3
2,Shannon Ross,Kleydson Rodrigues,Matt Wynne,"February 11, 2023","Perth, Western Australia, Australia",Kleydson Rodrigues,False,Flyweight,3
3,Justin Tafa,Parker Porter,Jason Herzog,"February 11, 2023","Perth, Western Australia, Australia",Justin Tafa,False,Heavyweight,3
4,Josh Culibao,Melsik Baghdasaryan,Peter Hickmott,"February 11, 2023","Perth, Western Australia, Australia",Josh Culibao,False,Featherweight,3
...,...,...,...,...,...,...,...,...,...
6961,Jason DeLucia,Scott Baker,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Jason DeLucia,False,Open Weight,1
6962,Remco Pardoel,Alberta Cerra Leon,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Remco Pardoel,False,Open Weight,1
6963,Frank Hamaker,Thaddeus Luster,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Frank Hamaker,False,Open Weight,1
6964,Scott Morris,Sean Daugherty,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Scott Morris,False,Open Weight,1


### Compiling Data per fighter

In [44]:
red_fighters = df['R_fighter'].value_counts().index
blue_fighters = df['B_fighter'].value_counts().index

fighters = list(set(red_fighters) | set(blue_fighters))

In [45]:
def get_renamed_winner(row):
    if row['R_fighter'] == row['Winner']:
        return 'Red'
    elif row['B_fighter'] == row['Winner']:
        return 'Blue'
    elif row['Winner'] == 'Draw':
        return 'Draw'

df2['Winner'] = df2[['R_fighter', 'B_fighter', 'Winner']].apply(get_renamed_winner, axis=1)

In [46]:
df = pd.concat([df,pd.get_dummies(df['win_by'], prefix='win_by')],axis=1)
df.drop(['win_by'],axis=1, inplace=True)

In [47]:
Numerical_columns = ['hero_KD', 'opp_KD', 'hero_SIG_STR_pct',
       'opp_SIG_STR_pct', 'hero_TD_pct', 'opp_TD_pct', 'hero_SUB_ATT', 'opp_SUB_ATT',
        'hero_REV', 'opp_REV', 'hero_SIG_STR._att', 'hero_SIG_STR._landed',
       'opp_SIG_STR._att', 'opp_SIG_STR._landed', 'hero_TOTAL_STR._att',
       'hero_TOTAL_STR._landed', 'opp_TOTAL_STR._att', 'opp_TOTAL_STR._landed',
       'hero_TD_att', 'hero_TD_landed', 'opp_TD_att', 'opp_TD_landed', 'hero_HEAD_att',
       'hero_HEAD_landed', 'opp_HEAD_att', 'opp_HEAD_landed', 'hero_BODY_att',
       'hero_BODY_landed', 'opp_BODY_att', 'opp_BODY_landed', 'hero_LEG_att',
       'hero_LEG_landed', 'opp_LEG_att', 'opp_LEG_landed', 'hero_DISTANCE_att',
       'hero_DISTANCE_landed', 'opp_DISTANCE_att', 'opp_DISTANCE_landed',
       'hero_CLINCH_att', 'hero_CLINCH_landed', 'opp_CLINCH_att', 'opp_CLINCH_landed',
       'hero_GROUND_att', 'hero_GROUND_landed', 'opp_GROUND_att', 'opp_GROUND_landed', 
        'hero_CTRL_time(seconds)', 'opp_CTRL_time(seconds)',
       'total_time_fought(seconds)']

Categorical_columns = ['win_by', 'last_round',
        'Winner', 'title_bout']

For all `Numerical_columns`, we take the average of those columns for every fighter of every fight they had up until that point.

For `Categorical_columns`, we have to come up with different ideas for each column:

* Each `win_by` will be a column of it's own
* from `last_round` we can get, `total_rounds_fought`
* from `total_time_fought` we can get `average_time_fought`
* from `Winner` we get `wins`, `losses`, `draw`, `current_streak`, `longest_streak`
* from `title_bout` we can get `no_of_title_fights`

In [48]:
import re

def lreplace(pattern, sub, string):
    """
    Replaces 'pattern' in 'string' with 'sub' if 'pattern' starts 'string'.
    """
    return re.sub('^%s' % pattern, sub, string)

In [49]:
red = df.groupby('R_fighter')
blue = df.groupby('B_fighter')

In [50]:
def get_fighter_red(fighter_name):
    try:
        fighter_red = red.get_group(fighter_name)
    except:
        return None
    rename_columns = {}
    for column in fighter_red.columns:
        if re.search('^R_', column) is not None:
            rename_columns[column] = lreplace('R_', 'hero_', column)
        elif re.search('^B_', column) is not None:
            rename_columns[column] = lreplace('B_', 'opp_', column)
    fighter_red = fighter_red.rename(rename_columns, axis='columns')
    return fighter_red

In [51]:
def get_fighter_blue(fighter_name):
    try:
        fighter_blue = blue.get_group(fighter_name)
    except:
        return None
    rename_columns = {}
    for column in fighter_blue.columns:
        if re.search('^B_', column) is not None:
            rename_columns[column] = lreplace('B_', 'hero_', column)
        elif re.search('^R_', column) is not None:
            rename_columns[column] = lreplace('R_', 'opp_', column)
    fighter_blue = fighter_blue.rename(rename_columns, axis='columns')
    return fighter_blue

In [52]:
def get_result_stats(result_list):
    result_list.reverse() # To get it in ascending order
    current_win_streak = 0
    current_lose_streak = 0
    longest_win_streak = 0
    wins = 0
    losses = 0
    draw = 0
    for result in result_list:
        if result == 'hero':
            wins += 1
            current_win_streak += 1
            current_lose_streak = 0
            if longest_win_streak < current_win_streak:
                longest_win_streak += 1
        elif result == 'opp':
            losses += 1
            current_win_streak = 0
            current_lose_streak += 1
        elif result == 'draw':
            draw += 1
            current_lose_streak = 0
            current_win_streak = 0
            
    return current_win_streak, current_lose_streak, longest_win_streak, wins, losses, draw

In [53]:
win_by_columns = ['win_by_Decision - Majority', 'win_by_Decision - Split',
       'win_by_Decision - Unanimous', 'win_by_KO/TKO','win_by_Submission',
       'win_by_TKO - Doctor\'s Stoppage']

In [54]:
temp_blue_frame = pd.DataFrame()
temp_red_frame = pd.DataFrame()
result_stats = ['current_win_streak', 'current_lose_streak', 'longest_win_streak', 'wins', 'losses', 'draw']

for fighter_name in fighters:
    fighter_red = get_fighter_red(fighter_name)
    fighter_blue = get_fighter_blue(fighter_name)
    fighter_index = None
    
    if fighter_red is None:
        fighter = fighter_blue
        fighter_index = 'blue'
    elif fighter_blue is None:
        fighter = fighter_red
        fighter_index = 'red'
    else:
        fighter = pd.concat([fighter_red, fighter_blue]).sort_index()
    
    fighter['Winner'] = fighter['Winner'].apply(lambda X: 'hero' if X == fighter_name else 'opp')

    for i, index in enumerate(fighter.index):
        fighter_slice = fighter[(i+1):].sort_index(ascending=False)
        s = fighter_slice[Numerical_columns].ewm(span=3, adjust=False).mean().tail(1)
        if len(s) != 0:
            pass
        else:
            s.loc[len(s)] = [np.NaN for _ in s.columns]
        s['total_rounds_fought'] = fighter_slice['last_round'].sum()
        s['total_title_bouts'] = fighter_slice[fighter_slice['title_bout']==True]['title_bout'].count()
        s['hero_fighter'] = fighter_name
        results = get_result_stats(list(fighter_slice['Winner']))
        for result_stat, result in zip(result_stats, results):
            s[result_stat] = result
        win_by_results = fighter_slice[fighter_slice['Winner'] == 'hero'][win_by_columns].sum()
        for win_by_column,win_by_result in zip(win_by_columns, win_by_results):
            s[win_by_column] = win_by_result
        s.index = [index]


        if fighter_index is None:
            if index in fighter_blue.index:
                temp_blue_frame = temp_blue_frame.append(s)
            elif index in fighter_red.index:
                temp_red_frame = temp_red_frame.append(s)
        elif fighter_index == 'blue':
            temp_blue_frame = temp_blue_frame.append(s)
        elif fighter_index == 'red':
            temp_red_frame = temp_red_frame.append(s)

In [55]:
temp_blue_frame.T

,5267,6039,1738,2088,4647,4767,6118,6185,6425,6501,...,6033,5684,5803,5890,6071,6252,6434,3961,4427,4553
hero_KD,0.3125,1,0,0,1,NaN,0.125,0.25,0,NaN,...,NaN,0.157227,0.314453,0.628906,0.515625,0.0625,0.5,0,0,NaN
opp_KD,0,0,0.5,0,0,NaN,0.5625,0.125,1,NaN,...,NaN,1.2832,1.56641,1.13281,0.53125,0.125,0,0.5,1,NaN
hero_SIG_STR_pct,0.391523,0.6775,0.45,0.47,0.42,NaN,0.520625,0.61125,0.92,NaN,...,NaN,0.264932,0.359863,0.379727,0.508906,0.475625,0.345,0.43,0.25,NaN
opp_SIG_STR_pct,0.454727,0.4725,0.41,0.4,0.43,NaN,0.5625,0.605,0.85,NaN,...,NaN,0.534619,0.449238,0.528477,0.403906,0.325625,0.235,0.495,0.45,NaN
hero_TD_pct,0.296406,0.33,0,0,1,NaN,0.459375,0.25875,0.5,NaN,...,NaN,0.0664062,0.132812,0.265625,0.0625,0.25,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
win_by_Decision - Split,1,0,0,0,0,0,0,0,0,0,...,0,2,2,2,1,0,0,0,0,0
win_by_Decision - Unanimous,3,1,0,0,1,0,0,0,0,0,...,0,2,2,2,2,2,1,0,0,0
win_by_KO/TKO,3,2,0,0,0,0,2,2,0,0,...,0,2,2,2,2,2,1,0,0,0
win_by_Submission,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Adding fighter details like height, weight, reach, stance and dob

In [56]:
fighter_details = fighter_details[fighter_details.index.isin(fighters)]
for col in fighter_details.columns:
    print(f"Number of NaN in {col} : {fighter_details[col].isnull().sum()}")

Number of NaN in Height : 13
Number of NaN in Weight : 10
Number of NaN in Reach : 641
Number of NaN in Stance : 74
Number of NaN in DOB : 141


In [57]:
def convert_to_cms(X):
    if X is np.NaN:
        return X
    elif len(X.split("'")) == 2:
        feet = float(X.split("'")[0])
        inches = int(X.split("'")[1].replace(' ', '').replace('"',''))
        return (feet * 30.48) + (inches * 2.54)
    else:
        return float(X.replace('"','')) * 2.54

In [58]:
fighter_details['Height_cms'] = fighter_details['Height'].apply(convert_to_cms)
fighter_details['Reach_cms'] = fighter_details['Reach'].apply(convert_to_cms)

In [59]:
fighter_details['Weight_lbs'] = fighter_details['Weight'].apply(lambda X: float(X.replace(' lbs.', '')) if X is not np.NaN else X)

In [86]:
fighter_details.columns

Index(['fighter_name', 'Stance', 'DOB', 'Height_cms', 'Reach_cms',
       'Weight_lbs'],
      dtype='object')

In [61]:
fighter_details.head()

,Height,Weight,Reach,Stance,DOB,Height_cms,Reach_cms,Weight_lbs
fighter_name,,,,,,,,
Danny Abbadi,"5' 11""",155 lbs.,NaN,Orthodox,"Jul 03, 1983",180.34,NaN,155.0
David Abbott,"6' 0""",265 lbs.,NaN,Switch,NaN,182.88,NaN,265.0
Hamdy Abdelwahab,"6' 2""",264 lbs.,"72""",Southpaw,"Jan 22, 1993",187.96,182.88,264.0
Daichi Abe,"5' 11""",170 lbs.,"71""",Orthodox,"Nov 27, 1991",180.34,180.34,170.0
Papy Abedi,"5' 11""",185 lbs.,NaN,Southpaw,"Jun 30, 1978",180.34,NaN,185.0


In [82]:
# i dont understand this step, there seems to be the wrong df selected to try this operations on,
# since pct_column dont seem to be present in "fighter_details" dataframe
pct_columns = ['Str_Acc','Str_Def', 'TD_Acc', 'TD_Def']

def pct_to_frac(X):
    if X != np.NaN:
        return float(X.replace('%', ''))/100
    else:
        return 0

for column in pct_columns:
    fighter_details[column] = fighter_details[column].apply(pct_to_frac)

KeyError: 'Str_Acc'

In [62]:
fighter_details.drop(['Height', 'Weight', 'Reach'], axis=1, inplace=True)

In [63]:
fighter_details.reset_index(inplace=True)
temp_red_frame.reset_index(inplace=True)
temp_blue_frame.reset_index(inplace=True)

In [64]:
temp_blue_frame = temp_blue_frame.merge(fighter_details, left_on='hero_fighter', right_on='fighter_name', how='left')
temp_blue_frame.set_index('index', inplace=True)

In [65]:
temp_blue_frame[['hero_fighter', 'fighter_name', 'Height_cms', 'Weight_lbs', 'DOB']].head(20)

,hero_fighter,fighter_name,Height_cms,Weight_lbs,DOB
index,,,,,
5267,Quinton Jackson,Quinton Jackson,185.42,205.0,"Jun 20, 1978"
6039,Quinton Jackson,Quinton Jackson,185.42,205.0,"Jun 20, 1978"
1738,Salim Touahri,Salim Touahri,177.80,170.0,"Sep 28, 1989"
2088,Salim Touahri,Salim Touahri,177.80,170.0,"Sep 28, 1989"
4647,Kazuki Tokudome,Kazuki Tokudome,180.34,155.0,"Mar 04, 1987"
4767,Kazuki Tokudome,Kazuki Tokudome,180.34,155.0,"Mar 04, 1987"
6118,Terry Martin,Terry Martin,172.72,185.0,"Oct 10, 1980"
6185,Terry Martin,Terry Martin,172.72,185.0,"Oct 10, 1980"
6425,Terry Martin,Terry Martin,172.72,185.0,"Oct 10, 1980"


In [66]:
temp_red_frame = temp_red_frame.merge(fighter_details, left_on='hero_fighter', right_on='fighter_name', how='left')
temp_red_frame.set_index('index', inplace=True)

In [67]:
temp_blue_frame.drop('fighter_name', axis=1, inplace=True)
temp_red_frame.drop('fighter_name', axis=1, inplace=True)

In [68]:
blue_frame = temp_blue_frame.add_prefix('B_')
red_frame = temp_red_frame.add_prefix('R_')

In [69]:
frame = blue_frame.join(red_frame, how='outer')

In [70]:
rename_cols = {}
for col in frame.columns:
    if 'hero' in col:
        rename_cols[col] = col.replace('_hero_', '_avg_').replace('.', '')
    if 'opp' in col:
        rename_cols[col] = col.replace('_opp_', '_avg_opp_').replace('.', '')
    if 'win_by' in col:
        rename_cols[col] = col.replace(' ', '').replace('-', '_').replace('\'s', '_')

In [71]:
frame.rename(rename_cols, axis='columns', inplace=True)

In [72]:
frame.drop(['R_avg_fighter','B_avg_fighter'], axis=1, inplace=True)

In [73]:
df2 = df2.join(frame, how='outer')

### Create Age

In [74]:
df2['R_DOB'] = pd.to_datetime(df2['R_DOB'])
df2['B_DOB'] = pd.to_datetime(df2['B_DOB'])
df2['date'] = pd.to_datetime(df2['date'])

In [75]:
def get_age(row):
    B_age = (row['date'] - row['B_DOB']).days
    R_age = (row['date'] - row['R_DOB']).days
    if np.isnan(B_age)!=True:
        B_age = math.floor(B_age/365.25)
    if np.isnan(R_age)!=True:
        R_age = math.floor(R_age/365.25)
    return pd.Series([B_age, R_age], index=['B_age', 'R_age'])

In [76]:
df2[['B_age', 'R_age']]= df2[['date', 'R_DOB', 'B_DOB']].apply(get_age, axis=1)

In [77]:
df2.drop(['R_DOB', 'B_DOB'], axis=1, inplace=True)

In [78]:
# df2.drop(df2.index[df2['Winner'] == 'draw'], inplace = True)

In [79]:
df2.to_csv('../data/data.csv', index=False)

In [80]:
df2

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_avg_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Jack Della Maddalena,Randy Brown,Marc Goddard,2023-02-11,"Perth, Western Australia, Australia",Red,False,Welterweight,3,0.570801,...,0,3,0,0,Switch,180.34,185.42,170.0,32.0,26.0
1,Tyson Pedro,Modestas Bukauskas,Steve Perceval,2023-02-11,"Perth, Western Australia, Australia",Blue,False,Light Heavyweight,3,0.000000,...,0,3,2,0,Orthodox,190.50,200.66,205.0,29.0,31.0
2,Shannon Ross,Kleydson Rodrigues,Matt Wynne,2023-02-11,"Perth, Western Australia, Australia",Blue,False,Flyweight,3,0.000000,...,0,0,0,0,Switch,167.64,167.64,125.0,27.0,33.0
3,Justin Tafa,Parker Porter,Jason Herzog,2023-02-11,"Perth, Western Australia, Australia",Red,False,Heavyweight,3,0.000000,...,0,2,0,0,Southpaw,182.88,187.96,264.0,37.0,29.0
4,Josh Culibao,Melsik Baghdasaryan,Peter Hickmott,2023-02-11,"Perth, Western Australia, Australia",Red,False,Featherweight,3,0.500000,...,1,0,0,0,Orthodox,177.80,185.42,145.0,31.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6961,Jason DeLucia,Scott Baker,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0,0,0,0,Southpaw,180.34,NaN,190.0,NaN,24.0
6962,Remco Pardoel,Alberta Cerra Leon,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0,0,0,0,Southpaw,190.50,NaN,260.0,NaN,NaN
6963,Frank Hamaker,Thaddeus Luster,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
6964,Scott Morris,Sean Daugherty,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0,0,0,0,Orthodox,177.80,NaN,210.0,18.0,NaN


In [85]:
for column in df2.columns:
    print(column)

R_fighter
B_fighter
Referee
date
location
Winner
title_bout
weight_class
no_of_rounds
B_avg_KD
B_avg_opp_KD
B_avg_SIG_STR_pct
B_avg_opp_SIG_STR_pct
B_avg_TD_pct
B_avg_opp_TD_pct
B_avg_SUB_ATT
B_avg_opp_SUB_ATT
B_avg_REV
B_avg_opp_REV
B_avg_SIG_STR_att
B_avg_SIG_STR_landed
B_avg_opp_SIG_STR_att
B_avg_opp_SIG_STR_landed
B_avg_TOTAL_STR_att
B_avg_TOTAL_STR_landed
B_avg_opp_TOTAL_STR_att
B_avg_opp_TOTAL_STR_landed
B_avg_TD_att
B_avg_TD_landed
B_avg_opp_TD_att
B_avg_opp_TD_landed
B_avg_HEAD_att
B_avg_HEAD_landed
B_avg_opp_HEAD_att
B_avg_opp_HEAD_landed
B_avg_BODY_att
B_avg_BODY_landed
B_avg_opp_BODY_att
B_avg_opp_BODY_landed
B_avg_LEG_att
B_avg_LEG_landed
B_avg_opp_LEG_att
B_avg_opp_LEG_landed
B_avg_DISTANCE_att
B_avg_DISTANCE_landed
B_avg_opp_DISTANCE_att
B_avg_opp_DISTANCE_landed
B_avg_CLINCH_att
B_avg_CLINCH_landed
B_avg_opp_CLINCH_att
B_avg_opp_CLINCH_landed
B_avg_GROUND_att
B_avg_GROUND_landed
B_avg_opp_GROUND_att
B_avg_opp_GROUND_landed
B_avg_CTRL_time(seconds)
B_avg_opp_CTRL_time(sec